<a href="https://colab.research.google.com/github/FurkatUZB/first_one/blob/main/ML_Modelni_trening_qilish_Linear_Regression_Chiziqli_regressiya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Science va Sun'iy Intellekt Praktikum
5-MODUL. Machine Learning
5.1 - ML loyiha qadamlari
Machine Learning
Nihoyat ma'lumotlarimiz ML uchun tayyor.

Boshlanishiga kerakli modullar va ma'lumotlarni qayta o'qib olamiz.

In [ ]:
import pandas as pd
import numpy as np
import sklearn # scikit-learn kutubxonasi

Ma'lumotlarni o'qiymiz

In [ ]:
# Onlayn dataset joylashgan manzilini ko'rsatamiaz
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

Ma'lumotlarni train va testga ajratamiz.

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_set.drop("median_house_value", axis=1)
y = train_set["median_house_value"].copy()

X_num = X_train.drop("ocean_proximity", axis=1)

Pipeline quramiz

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

Sonli ustunlar uchun

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('attribs_adder', CombinedAttributesAdder(add_bedrooms_per_room = True)),
          ('std_scaler', StandardScaler())
])

Matnli ustunlar uchun

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

Mana yakuniy, to'liq konveyer tayyor bo'ldi (full_pipeline).

Konveyerni ishga tushirish uchun .fit_transform() metodini chaqrisih kifoya.

In [ ]:
X_prepared = full_pipeline.fit_transform(X_train)

In [ ]:
X_prepared[0:5,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        -0.2117846 ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.34218528,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
        -0.66165785,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.23269811, -1.38217186,  0.58654547, -0.56148971, -0.40930582,
        -0.00743434, -0.38058662, -1.01786438, -0.60001532,  0.07750687,
         0.78303162,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.10855122,  0.5320839 ,  1.14200767, -0.11956547, -0.25655915,
        -0.48587717, -0.31496232, -0.17148831,  0.3490073 , -0.06883176,
        -0.55036364,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ]])
Ma'lumotlar ML uchun tayyor.

Machine Learning
Bizning maqsadimiz bashorat qilish, buning uchun bir nechta ML algoritmlar mavjud.

Biz kelgusi darslarda ularning har biri bilan yaqinda tanishamiz, hozir esa scikit-learn tarkibidagi ba'zi tayyor algoritmlardan foydalanamiz.

Linear Regression - Chiziqli regressiya
sklearn tarkibidagi LinearRegression klassidan yangi model yaratamiz.

In [ ]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

LinearRegression bu estimator. Estimatorlar ma'lumotlarni qabul qilib oladi va .fit() metodi yordamida ulardan basorat qilishni o'rganadi (machine learning)

In [ ]:
LR_model.fit(X_prepared, y)

LinearRegression()
TAMOM! Machine Learning tugadi! Ha, adashmadingiz, bor yo'g'i 3 qator kod bilan biz hozirgina kompyuterga uylarni narxini bashorat qilishni o'rgatdik.

Modelni qanday qilib tekshirib ko'rishimiz mumkin? Keling housing datasetdan biror qatorni modelga beramiz va chiqqan natijani bizdagi bor natija (label) bilan solishtiramiz.

In [ ]:
# tasodifiy 5 ta qatorni ajratib olamiz
test_data = X_train.sample(5)
test_data

In [ ]:
# yuqoridagi qatorlarga mos keluvchi narxlarni ajratib olamiz (biz aynan shu qiymatlarni bashorat qilishimiz kerak)
test_label = y.loc[test_data.index]
test_label

6211     218800.0
14295    199100.0
12320     61000.0
19065    173300.0
16727    250600.0
Name: median_house_value, dtype: float64
test_data ni pipelinedan o'tkazib, bizga kerak ko'rinishga keltirib olamiz.

Ahamiyat bering bu safgar biz .transform() metodini chaqiramiz, sababi .fit() metodini avval chaqirgan edik.

In [ ]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

array([[ 0.84378566, -0.73628606,  0.50719373, -0.83142678, -0.95823916,
        -0.90979159, -0.98958015,  0.33459385,  0.56350326,  0.00777575,
        -0.83784003,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.22272599, -1.36813087,  1.85617335, -0.49756934, -0.55250582,
        -0.58877756, -0.4855855 , -0.6157084 , -0.20225927, -0.0599208 ,
        -0.27698044,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.53186153, -0.82053203, -0.60373066, -0.62449035, -0.55489249,
        -0.78226547, -0.70083322, -1.01392579,  0.03162339, -0.06842401,
         0.43900995,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [-1.43484237,  1.23881402, -0.60373066, -0.10071126,  0.05132085,
        -0.2888713 ,  0.00790925, -0.7965158 , -0.25892721, -0.0789452 ,
         0.31487031,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [-0.58721269, -0.05763792, -1.71465506,  3.20567298,  2.69097422,
         2.691722  ,  3.02662719, -0.11240951,  0.15928101, -0.0330359 ,
        -0.68198853,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])
Bashorat (predict) qilamiz:

In [ ]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data

array([228852.74349408, 203387.28874123,  64236.23414498, 168843.70532715,
       241650.14389617])
Yuoqirda ko'rib turganingiz bashorat qilingan qiymatlar. Xo'sh, ular real qiymatlardan qanday farq qiladi, solishtiramiz:

In [ ]:
pd.DataFrame({'Prognoz':predicted_data, 'Real baxosi': test_label})


5-QADAM. Modelni baholaymiz
Ko'rib turganingizdek, modelimiz qayerdadur kamroq, qayeradur ko'proq xato bilan bashorat qilgan.
Lekin model aniqligini baxolash uchun 5 qator yetarli emas. Keling, avvalroq ajratib olgan test set yordamida sinab ko'ramiz:

In [ ]:
test_set

Avval predictor ustunlarni ajratib olaylik (median_house_value) dan boshqa barcha ustunlar):

In [ ]:
X_test = test_set.drop('median_house_value', axis=1)
X_test

Label (median_house_value) ustunini ajratib olamiz.

In [ ]:
y_test = test_set['median_house_value'].copy()
y_test

Name: median_house_value, Length: 4128, dtype: float64
test_set ni ham pipelinedan o'tkazamiz:

In [ ]:
X_test_prepared = full_pipeline.transform(X_test)

Bashorat

In [ ]:
y_predicted = LR_model.predict(X_test_prepared)

Bashorat va real datani solishtirish uchun avvalgi bo'limda ko'rgan Root mean square error (RMSE) dan foydalanamiz:

In [ ]:
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

72701.32600762139
Demak, RMSE=72701$ chiqdi. Yomon emas, lekin yaxshi ham emas. Ya'ni modelimiz uylarni baholashda o'rtacha 72000$ ga adashayapti.

Model aniqligini oshirish uchun yagona, universal yechim yo'q. Qilib ko'rishingiz mumkin bo'lgan ishlar:

Yaxhsiroq paramterlar topish
Yaxhsiroq model (algoritm) tanlash
Ko'proq ma'lumot yig'ish va hokazo.
Biz hozir boshqa model bilan sinab ko'ramiz.

DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
Tree_model = DecisionTreeRegressor()
Tree_model.fit(X_prepared, y)

DecisionTreeRegressor()
Modelni tekshiramiz:

In [ ]:
y_predicted = Tree_model.predict(X_test_prepared)

In [ ]:
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

72562.00852508846
Avvalgidan katta farq qilmadi.

RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()
Modelni tekshiramiz:

In [ ]:
y_predicted = RF_model.predict(X_test_prepared)
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

50331.07889872079
Avvalgidan yaxhsiroq.

Cross-Validation usuli bilan baholash
Yuqorida biz modelni baholash uchun ma'lumotlarni test va train setlarga ajratdik.
Bu usulning kamchiligi biz test va train uchun doim bir xil ma'lumotlardan foydalanayapmiz.

Cross-validation yordamida biz ma'lumotlarni bir necha qismga ajratib, modelni turli qismlar yordamida bir nechta bor train va test qilishimiz mumkin.

Misol uchun, quyidagi rasmda ma'lumotlarni 5 ga ajratib train va test qilish ko'rsatilgan.

alt

Cross validation uchun ma'lumotlarni train va testga bo'lish shart emas, buni sklearn o'zi qiladi.

In [ ]:
X = df.drop("median_house_value", axis=1)
y = df["median_house_value"].copy()

X_prepared = full_pipeline.transform(X)

Validation natijalarini ko'rsatish uchun sodda funksiya yasab olamiz

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Std.dev:", scores.std())

Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score

LogisticRegression

In [ ]:
scores = cross_val_score(LR_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)

In [ ]:
display_scores(LR_rmse_scores)

Scores: [73394.92502922 74814.24096819 75431.93119241 76608.78768825
 66196.48128669]
Mean: 73289.27323295096
Std.dev: 3694.713678722354
Decision Tree

In [ ]:
scores = cross_val_score(Tree_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [117426.24650262  71236.93347566  84784.96821511  74123.27730087
  89750.15887077  76806.63059624  69589.05516706 102133.30525079
  95246.44610184  75049.59397392]
Mean: 85614.66154548783
Std.dev: 14765.492299059766
Random Forest

In [ ]:
scores = cross_val_score(RF_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [96963.10192753 47862.65729636 64981.93435835 56938.85521984
 60802.46090989 60535.50319764 46989.4400147  79484.26918866
 74040.3414757  49323.01936423]
Mean: 63792.158295289555
Std.dev: 15049.727081731291
Modelni saqlash
Yaratgan modelimizdan kelajakda foydalanish uchun saqlab qo'yishimiz lozim. Umuman olganda nafaqat model, balki boshqa kerak bo'ladigan o'zgaruvchilarni ham saqlab qo'yish maqsadga muvvofiq bo'ladi. Masalan pipeline.

Buning uchun Pythondagi pickle yoki joblib modullaridan foydalanamiz.

pickle yordamida saqlash

In [ ]:
import pickle

filename = 'RF_model.pkl' # faylga istalgan nom beramiz
with open(filename, 'wb') as file:
    pickle.dump(RF_model, file)

Modelni qayta o'qiymiz:

In [ ]:
with open(filename, 'rb') as file:
    model = pickle.load(file)

Modelni sinab ko'ramiz

In [ ]:
scores = cross_val_score(model, X_prepared, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [76978.93742397 64329.3404101  61109.46641995 81775.30419816
 62027.72651085]
Mean: 69244.15499260693
Std.dev: 8476.579030000677
joblib yordamida saqlash
joblib katta NumPy martrisalarni siqib saqlash uchun afzal.

joblib o'rnatilmagan bo'lsa pip install joblib yordamida o'rnatib oling.

In [ ]:
import joblib

filename = 'RF_model.jbl' # faylga istalgan nom beramiz
joblib.dump(RF_model, filename)

['RF_model.jbl']
Modelni o'qiymiz

In [ ]:
model = joblib.load(filename)

In [ ]:
scores = cross_val_score(model, X_prepared, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [77460.56831558 63838.3153173  61358.53227924 80371.60644565
 62380.21986169]
Mean: 69081.84844389423
Std.dev: 8120.562299609714
pipeline saqlab olamiz

In [ ]:
filename = 'pipeline.jbl'
joblib.dump(full_pipeline, filename)

['pipeline.jbl']